In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import json
from datetime import datetime
from datetime import timedelta
from datetime import date
import locale

In [4]:
folder = 'C:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data'

def htm_to_csv(html_file, csv_file, id):
    with open(html_file, encoding='utf8') as infile:
        soup = BeautifulSoup(infile, "html.parser")

        csv_file = pd.read_csv(csv_file)

        df = pd.DataFrame()

        offers_text = soup.find(class_='css-1it8m2y').get_text()
        n_offers = int(offers_text.split()[0])


        df['insurance_company'] = [x.get_text() for x in soup.find_all(class_='css-1mpoat2')][:n_offers]

        df['insurance_model'] = [x.get_text() for x in soup.find_all(class_='css-1no210n')][:n_offers]

        df['user_rating'] = [x.get_text() for x in soup.find_all(class_='css-1r2y0ft')][:n_offers]

        df['price'] = [x.get_text() for x in soup.find_all(class_='css-vibb24')][:n_offers]

        df['date_of_birth'] = csv_file['date_of_birth'][0]
        df['date_of_drivers_license'] = csv_file['date_of_drivers_license'][0]
        df['gender'] = csv_file['gender'][0]
        df['nationality'] = csv_file['nationality'][0]
        df['id'] = id

        df.set_index('id', inplace=True)

        return(df)

def get_all(rootdir):
    
    counter = 0
    master_df = pd.DataFrame()

    for subdir, dirs, files in os.walk(rootdir):
        html = ''
        csv = ''
        for file in files:
            file_path = os.path.join(subdir, file)
            if file_path.endswith('.htm') or file_path.endswith('.html'):
                html = file_path
            elif file_path.endswith('.csv'):
                csv = file_path

        if html != '' and csv != '':
            # print(csv)
            # print(html)
            counter += 1
            try:
                master_df = pd.concat([master_df, htm_to_csv(html, csv, counter)], axis=0)
            except ValueError:
                print("Error at" + html)
            except:
                print("Another error has occured")
            
            html = ''
            csv = ''
    print(counter)
    return(master_df)

master_df = get_all(folder)

today = date.today()

age_list = []
for dob in master_df['date_of_birth']:
    dob = datetime.strptime(dob, '%Y-%m-%d').date()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    age_list.append(age)

master_df['age'] = age_list

dsd_list = []
for dod in master_df['date_of_drivers_license']:
    dod = datetime.strptime(dod, '%Y-%m-%d').date()
    age = today.year - dod.year - ((today.month, today.day) < (dod.month, dod.day))
    dsd_list.append(age)

master_df['time_since_dl'] = dsd_list

price_floats = []

for raw_price in master_df['price']:
    locale.setlocale(locale.LC_ALL, 'de_CH.UTF8')
    conv = locale.localeconv()
    raw_numbers = raw_price.strip(conv['currency_symbol'])
    raw_numbers = raw_numbers.replace("'", "")
    amount = locale.atof(raw_numbers)
    price_floats.append(amount)

master_df['price'] = price_floats

#htm_to_csv('html_files/test_html.htm')
master_df.to_csv('master_df.csv')

anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_1\Stephan 01.html
anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_10\Stephan 10.html
anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_12\Stephan 12.html
anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_13\Stephan 13.html
anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_14\Stephan 14.html
anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_15\Stephan 15.html
anoher error atC:/Users/samra/OneDrive/Dokumente/Universität/HS 22/Markets and Norms/Data/search_data\Stephan\search_16\Stephan 16.html
anoher error atC:/Users/samra/OneDrive/Dokumente/

In [16]:
def get_metadata(html_file):
    with open(html_file, encoding='utf8') as infile:
        soup = BeautifulSoup(infile, "html.parser")

        metadata = soup.find(id='__NEXT_DATA__')

        meta = metadata.get_text()
        meta = json.loads(meta)
        meta_text = meta['props']['pageProps']['response']['InputData']
        car_brand = meta_text['VehicleData']['CarMake']
        car_model = meta_text['VehicleData']['CarModel']
        car_type = meta_text['VehicleData']['CarType']

        gender = meta_text['Driver']['Gender']
        date_of_birth = meta_text['Driver']['DateOfBirth']
        drivingslicense_date = meta_text['Driver']['DrivingLicenseDate']
        zip = meta_text['Driver']['PlaceOfResidence']

        

get_metadata('html_files/test_html.htm')

In [3]:
# Converte xlsx into csv

from pathlib import Path

rootdir = 'search_data/Stephan/'

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        try:
            file_path = os.path.join(subdir, file)
            if file_path.endswith('.xlsx'):
                print(file_path)

                df_pre = pd.read_excel(file_path)
                p_pre = Path(file_path)
                df_pre.to_csv(file_path.replace('.xlsx', '.csv'), index = None)

                p = Path(p_pre.with_suffix('.csv'))
                p.rename(p.with_suffix('.txt'))

                path_txt = p_pre.with_suffix('.txt')

                with open(path_txt, 'r') as infile:
                    path_new = str(path_txt).replace('.txt', '_new.txt')
                    with open(path_new, 'w') as outfile:
                        for line in infile:
                            outfile.write(line.replace('"', ''))

                df_new = pd.read_csv(path_new)
                df_new.to_csv(p_pre.with_suffix('.csv'), index=None)
        except:
            print("Error at:" + file_path)
        

search_data/Stephan/search_0\search_params0.xlsx
search_data/Stephan/search_1\search_params1 1.xlsx
search_data/Stephan/search_1\search_params1.xlsx
Error at:search_data/Stephan/search_1\search_params1.xlsx
search_data/Stephan/search_10\search_params10 1.xlsx
search_data/Stephan/search_10\search_params10.xlsx
Error at:search_data/Stephan/search_10\search_params10.xlsx
search_data/Stephan/search_11\search_params11.xlsx
search_data/Stephan/search_12\search_params12 1.xlsx
search_data/Stephan/search_12\search_params12.xlsx
Error at:search_data/Stephan/search_12\search_params12.xlsx
search_data/Stephan/search_13\search_params13 1.xlsx
search_data/Stephan/search_13\search_params13-DESKTOP-098J75F.xlsx
Error at:search_data/Stephan/search_13\search_params13-DESKTOP-098J75F.xlsx
search_data/Stephan/search_13\search_params13.xlsx
Error at:search_data/Stephan/search_13\search_params13.xlsx
search_data/Stephan/search_14\search_params14 1.xlsx
search_data/Stephan/search_14\search_params14.xlsx
Err

In [34]:



df_pre = pd.read_excel('search_data/Stephan/search_2/search_params2.xlsx')
df_pre.to_csv('search_data/Stephan/search_2/search_params2.csv', index = None)

p = Path('search_data/Stephan/search_2/search_params2.csv')
p.rename(p.with_suffix('.txt'))
with open('search_data/Stephan/search_2/search_params2.txt', 'r') as infile:
    with open('search_data/Stephan/search_2/search_params2_new.txt', 'w') as outfile:
        for line in infile:
            outfile.write(line.replace('"', ''))

df_test = pd.read_csv('search_data/Stephan/search_2/search_params2_new.txt')
df_test

,date_of_birth,date_of_drivers_license,gender,nationality
0,1975-08-18,2005-06-20,f,Frankreich


In [36]:
p = Path('search_data/Stephan/search_2/search_params2.xlsx')
print(p.with_suffix('.txt'))

ValueError: Invalid suffix 'new.txt'